In [1]:
import tensorflow as tf
import random
from os import walk
import pandas as pd

In [12]:
model_dir = '../animal_classifier/models/'
models = ['augmented', 'not_augmented']
epochs = [0,16,32,48,64]

train_dir = '../animal_classifier/dataset/train'
validate_dir = '../animal_classifier/dataset/validate'
data_output_dir = './data_output'

epsilons = [0, .1, .2, .3, .4]
sample_size = 16 # Samples per class
image_dim = 256

In [3]:
classes = ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
validate_folder_size = 200

In [4]:
def generate_indices(): #Optional code that generates random images to use
    index_matrix = []

    for animal in classes:
        indices = random.sample(range(validate_folder_size), sample_size)
        index_matrix.append(indices)
    return index_matrix


In [5]:
# print(generate_indices())
index_matrix = [[53, 31, 107, 90, 75, 140, 85, 160, 177, 158, 68, 154, 6, 54, 8, 163], [18, 119, 160, 124, 78, 108, 110, 55, 188, 173, 82, 67, 114, 7, 40, 128], [172, 25, 109, 67, 145, 117, 143, 134, 138, 148, 10, 86, 189, 7, 166, 89], [69, 111, 67, 112, 170, 40, 146, 95, 64, 178, 17, 193, 104, 197, 21, 161], [174, 114, 170, 158, 84, 65, 173, 126, 77, 188, 194, 145, 2, 33, 182, 135], [197, 164, 105, 88, 64, 111, 3, 159, 28, 135, 80, 96, 14, 35, 173, 172], [46, 165, 199, 39, 194, 26, 153, 131, 120, 34, 172, 130, 90, 5, 157, 17], [0, 176, 73, 84, 155, 143, 119, 174, 83, 95, 132, 39, 162, 90, 100, 62], [78, 108, 8, 107, 139, 87, 29, 188, 4, 112, 51, 42, 33, 28, 93, 192], [101, 46, 15, 76, 154, 86, 112, 120, 78, 74, 68, 107, 153, 149, 80, 87]]
#Which image indices for each class

In [6]:
# Generates image paths
def get_image(animal, index):
    directory = validate_dir + '/' + animal
    filenames = next(walk(directory), (None, None, []))[2] #Black magic that returns list of filenames in directory
    return tf.io.read_file(directory + '/' + filenames[index])

In [7]:
def get_perterbations(model, input_image, class_index):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model(input_image)
    loss = tf.keras.losses.MSE(class_index, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [8]:
def get_loss(probs, animal_index): #Calculates MSE loss
    total = 0
    for index, prob in enumerate(probs):
        if index == animal_index:
            total += (1-prob)*(1-prob)
        else:
            total += prob*prob
    return total/len(probs)

In [9]:
def create_adversarial_dataset(model):    
    eps_all_losses = [[] for __ in epsilons] # [[]] * len(epsilons) makes a list of lists that reference eachother. Editing one affects all of them.
    eps_losses = [0] * len(epsilons)
    eps_accuracy = [0] * len(epsilons)

    for animal_index, indices in enumerate(index_matrix):
        animal_name = classes[animal_index]
        
        for index in indices:
            # Load and process image
            image_raw = get_image(animal_name, index)
            image = tf.image.decode_image(image_raw, channels=3)
            image = tf.cast(image, tf.float32)
            image = tf.image.resize(image, (256,256))
            image = image[None, ...] #adds that goofy empty dimension

            perterbations = get_perterbations(model, image, animal_index)

            # Check new predicted value for each epsilon
            for eps_index, eps in enumerate(epsilons):
                image_probs = model.predict(image - perterbations * eps)[0]

                loss = get_loss(image_probs, animal_index)
                eps_losses[eps_index] += loss
                eps_all_losses[eps_index].append(loss)

                # If correct guess -> add one to total
                if image_probs[animal_index] == max(image_probs):
                    eps_accuracy[eps_index] += 1

    eps_avg_losses = [total / (len(classes) * sample_size) for total in eps_losses]
    eps_avg_accuracy = [total / (len(classes) * sample_size) for total in eps_accuracy]

    return (eps_avg_losses, eps_avg_accuracy, eps_all_losses)


In [13]:
for model_name in models:
    row_list = []
    for epoch in epochs:
        model = tf.keras.models.load_model(model_dir + model_name + '/' + model_name + '_' + str(epoch))
        [avg_loss, avg_accuracy, all_losses] = create_adversarial_dataset(model)
        row_list.append([epoch, *avg_loss, *avg_accuracy])
        loss_df = pd.DataFrame(all_losses, index = [*[str(eps) for eps in epsilons]])
        loss_df = loss_df.transpose()
        loss_df.to_csv(data_output_dir + '/all_losses/' + model_name + '_' + str(epoch) + '.csv', index=False)
        print(model_name + '_' + str(epoch) + '.csv created!')

    general_df = pd.DataFrame(row_list, columns=['Epoch', *['Loss_' + str(eps) for eps in epsilons], *['Accuracy_' + str(eps) for eps in epsilons]])
    general_df.to_csv(data_output_dir + '/general/' + model_name + '.csv', index=False)
    print(model_name + '.csv created!')

augmented_0.csv created!
augmented_16.csv created!
augmented_32.csv created!
augmented_48.csv created!
augmented_64.csv created!
augmented.csv created!
not_augmented_0.csv created!
not_augmented_16.csv created!
not_augmented_32.csv created!
not_augmented_48.csv created!
not_augmented_64.csv created!
not_augmented.csv created!
